# Quantitative Momentum Strategy

"Momentum investing" 

Building an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports


In [1]:
import pandas as pd
import numpy as np
import math
import xlsxwriter
from scipy import stats
import yfinance as yf
from datetime import datetime, timedelta

## Importing Our List of Stocks
Same as previous file

In [2]:
stocks = pd.read_csv('sp_500_stocks_new.csv')

## Use of Y-Finance

In [3]:
data = yf.Ticker('aapl')
data.info
data.history(period='1m')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-05-17 00:00:00-04:00,189.380005,190.809998,189.220001,189.869995,40771639,0.0,0.0


In [4]:
ticker_symbol = "AAPL"
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Download historical data
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate percentage change
start_price = stock_data['Adj Close'].iloc[0]
end_price = stock_data['Adj Close'].iloc[-1]

percentage_change = ((end_price - start_price) / start_price) * 100
percentage_change

[*********************100%%**********************]  1 of 1 completed


8.39803102736529

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Adding our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)


for symbol_string in symbol_strings:
    batch_api_call_url = yf.Ticker(symbol_string)
    for symbol in symbol_string.split(','):
        try:
            data = yf.Ticker(symbol)
            price = data.info['currentPrice']
            # Download historical data
            end_date = datetime.now()
            start_date = end_date - timedelta(days=365)
            
            # Download historical data
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            
            # Calculate percentage change
            start_price = stock_data['Adj Close'].iloc[0]
            end_price = stock_data['Adj Close'].iloc[-1]
            
            percentage_change = ((end_price - start_price) / start_price)

            
            new_data = pd.Series([symbol, price, percentage_change,'N/A'], index= my_columns)
            final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)
        except:
            print(data,' not found')
final_dataframe

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

yfinance.Ticker object <BRK.B>  not found



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

yfinance.Ticker object <BF.B>  not found



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,105.26,0.344753,N/A
1,AOS,85.68,0.273214,N/A
2,ABT,104.09,-0.025059,N/A
3,ABBV,166.42,0.193407,N/A
4,ACN,303.59,0.063302,N/A
...,...,...,...,...
498,ZBRA,317.89,0.132087,N/A
499,ZBH,120.18,-0.102895,N/A
500,ZTS,174.06,-0.021126,N/A
501,MMM,105.26,0.344753,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [19]:
final_dataframe.sort_values('One-Year Price Return', ascending= False, inplace= True)
final_dataframe_top50 = final_dataframe[:50]
final_dataframe_top50.reset_index(inplace=True)
final_dataframe_top50

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,427,SMCI,887.89,4.407698,N/A
1,474,VST,94.08,2.929499,N/A
2,347,NVDA,924.79,1.958951,N/A
3,124,CEG,213.11,1.607484,N/A
4,345,NRG,82.3,1.537802,N/A
5,44,ANET,319.89,1.222848,N/A
6,74,AVGO,1395.29,1.085573,N/A
7,461,URI,685.79,0.987181,N/A
8,490,WRK,52.94,0.962472,N/A
9,141,DECK,888.56,0.953781,N/A


## Calculating the Number of Shares to Buy

In [26]:
def portfolio_input():
    global portfolio_size
    while True:
        try:
            portfolio_size = input('Enter the value of your portfolio: ')
            portfolio_size = float(portfolio_size)
            break
        except ValueError:
            print('Invalid input. Enter an integer please. Try Again.')
portfolio_input()
portfolio_size

Enter the value of your portfolio:  10_000_000


10000000.0

In [31]:
position_size = portfolio_size / len(final_dataframe_top50)
for i in range(len(final_dataframe_top50)):
    final_dataframe_top50.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe_top50.loc[i, 'Price'])
final_dataframe_top50

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,427,SMCI,887.89,4.407698,225
1,474,VST,94.08,2.929499,2125
2,347,NVDA,924.79,1.958951,216
3,124,CEG,213.11,1.607484,938
4,345,NRG,82.3,1.537802,2430
5,44,ANET,319.89,1.222848,625
6,74,AVGO,1395.29,1.085573,143
7,461,URI,685.79,0.987181,291
8,490,WRK,52.94,0.962472,3777
9,141,DECK,888.56,0.953781,225


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [36]:
def oneYearPercentageChange(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    start_price = stock_data['Adj Close'].iloc[0]
    end_price = stock_data['Adj Close'].iloc[-1]
    percentage_change = ((end_price - start_price) / start_price)
    return percentage_change

def sixMonthPercentageChange(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=182)
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    start_price = stock_data['Adj Close'].iloc[0]
    end_price = stock_data['Adj Close'].iloc[-1]
    percentage_change = ((end_price - start_price) / start_price)
    return percentage_change

def threeMonthPercentageChange(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=91)
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    start_price = stock_data['Adj Close'].iloc[0]
    end_price = stock_data['Adj Close'].iloc[-1]
    percentage_change = ((end_price - start_price) / start_price)
    return percentage_change
    
def oneMonthPercentageChange(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    start_price = stock_data['Adj Close'].iloc[0]
    end_price = stock_data['Adj Close'].iloc[-1]
    percentage_change = ((end_price - start_price) / start_price)
    return percentage_change

In [86]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One year Price Returns',
    'One year Return Percentile',
    'Six month Price Returns',
    'Six month Return Percentile',
    'Three month Price Returns',
    'Three month Return Percentile',
    'One month Price Returns',
    'One month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = yf.Ticker(symbol_string)
    for symbol in symbol_string.split(','):
        try:
            data = yf.Ticker(symbol)
            price = data.info['currentPrice']
            number_of_shares = 'N/A'
            one_year_price_returns = oneYearPercentageChange(symbol)
            one_year_return_percentile = 'N/A'
            six_month_price_returns = sixMonthPercentageChange(symbol)
            six_month_return_percentile = 'N/A'
            three_month_price_return = threeMonthPercentageChange(symbol)
            three_month_return_percentile = 'N/A'
            one_month_price_return = oneMonthPercentageChange(symbol)
            one_month_return_percentile = 'N/A'
            new_data = pd.Series([symbol, price, number_of_shares,one_year_price_returns,one_year_return_percentile,six_month_price_returns,six_month_return_percentile,three_month_price_return,three_month_return_percentile,one_month_price_return,one_month_return_percentile, 'N/A'], index= hqm_columns)
            hqm_dataframe = pd.concat([hqm_dataframe, new_data.to_frame().T], ignore_index=True)
        except:
            continue

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [87]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One year Price Returns,One year Return Percentile,Six month Price Returns,Six month Return Percentile,Three month Price Returns,Three month Return Percentile,One month Price Returns,One month Return Percentile,HQM Score
0,MMM,105.26,N/A,0.271241,N/A,0.351864,N/A,0.371063,N/A,0.150634,N/A,N/A
1,AOS,85.68,N/A,0.273214,N/A,0.134531,N/A,0.079995,N/A,-0.007835,N/A,N/A
2,ABT,104.09,N/A,-0.025059,N/A,0.038652,N/A,-0.10317,N/A,-0.011209,N/A,N/A
3,ABBV,166.42,N/A,0.193407,N/A,0.22608,N/A,-0.044244,N/A,0.010689,N/A,N/A
4,ACN,303.59,N/A,0.063302,N/A,-0.075617,N/A,-0.16031,N/A,-0.039272,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,ZBRA,317.89,N/A,0.132087,N/A,0.441417,N/A,0.169014,N/A,0.164304,N/A,N/A
499,ZBH,120.18,N/A,-0.109778,N/A,0.068267,N/A,-0.038406,N/A,0.007123,N/A,N/A
500,ZTS,174.06,N/A,-0.030576,N/A,-0.01136,N/A,-0.066953,N/A,0.13683,N/A,N/A
501,MMM,105.26,N/A,0.271241,N/A,0.325447,N/A,0.371063,N/A,0.150634,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [116]:
time_periods = [
    'One year',
    'Six month',
    'Three month',
    'One month'
]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_column = f'{time_period} Price Returns'
        percentile_column = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_column] = stats.percentileofscore(hqm_dataframe[change_column], hqm_dataframe.loc[row, change_column])/100

In [117]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One year Price Returns,One year Return Percentile,Six month Price Returns,Six month Return Percentile,Three month Price Returns,Three month Return Percentile,One month Price Returns,One month Return Percentile,HQM Score
474,VST,94.08,212,2.838433,0.998012,1.712021,0.998012,1.03857,1.0,0.428051,1.0,99.900596
345,NRG,82.3,243,1.45745,0.992048,0.740326,0.992048,0.591876,0.996024,0.167045,0.93837,97.962227
124,CEG,213.11,93,1.607484,0.994036,0.746228,0.994036,0.620097,0.998012,0.159088,0.924453,97.763419
237,HWM,82.93,241,0.844529,0.966203,0.610917,0.982107,0.314055,0.970179,0.305984,0.99006,97.713718
44,ANET,319.89,62,1.222848,0.99006,0.485511,0.970179,0.232195,0.924453,0.243789,0.984095,96.719682
...,...,...,...,...,...,...,...,...,...,...,...,...
482,WBD,8.05,N/A,-0.340164,0.013917,-0.250466,0.013917,-0.186047,0.017893,-0.031288,0.137177,4.572565
80,CZR,35.52,N/A,-0.188856,0.063618,-0.232332,0.017893,-0.147179,0.045726,-0.064771,0.055666,4.572565
136,CVS,57.68,N/A,-0.138691,0.083499,-0.13907,0.04175,-0.236822,0.011928,-0.160713,0.003976,3.528827
422,SBUX,77.85,N/A,-0.262155,0.029821,-0.253595,0.011928,-0.167023,0.027833,-0.106713,0.017893,2.186879


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [118]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [119]:
hqm_dataframe.sort_values('HQM Score', ascending= False, inplace= True)
hqm_dataframe_top50 = hqm_dataframe[:50]
hqm_dataframe_top50.reset_index()
hqm_dataframe_top50

,Ticker,Price,Number of Shares to Buy,One year Price Returns,One year Return Percentile,Six month Price Returns,Six month Return Percentile,Three month Price Returns,Three month Return Percentile,One month Price Returns,One month Return Percentile,HQM Score
474,VST,94.08,212,2.838433,0.998012,1.712021,0.998012,1.03857,1.0,0.428051,1.0,0.999006
345,NRG,82.3,243,1.45745,0.992048,0.740326,0.992048,0.591876,0.996024,0.167045,0.93837,0.979622
124,CEG,213.11,93,1.607484,0.994036,0.746228,0.994036,0.620097,0.998012,0.159088,0.924453,0.977634
237,HWM,82.93,241,0.844529,0.966203,0.610917,0.982107,0.314055,0.970179,0.305984,0.99006,0.977137
44,ANET,319.89,62,1.222848,0.99006,0.485511,0.970179,0.232195,0.924453,0.243789,0.984095,0.967197
388,QCOM,193.86,103,0.831287,0.962227,0.496873,0.976143,0.27573,0.95825,0.200818,0.960239,0.964215
207,GRMN,170.94,117,0.684392,0.93837,0.438988,0.93837,0.399473,0.988072,0.214753,0.974155,0.959742
34,APH,131.91,151,0.756417,0.956262,0.479281,0.968191,0.263513,0.94831,0.189665,0.952286,0.956262
478,WAB,168.32,118,0.727954,0.952286,0.455804,0.95825,0.252381,0.946322,0.166701,0.936382,0.94831
316,MU,125.29,159,0.837905,0.964215,0.593615,0.980119,0.552348,0.994036,0.11936,0.817097,0.938867


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [121]:
portfolio_input()

Enter the value of your portfolio:  1_000_000


In [122]:
position_size = float(portfolio_size) / len(hqm_dataframe_top50.index)
for i in hqm_dataframe_top50.index:
    hqm_dataframe_top50.loc[i , 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe_top50.loc[i, 'Price'])
hqm_dataframe_top50.reset_index(drop= True)

/var/folders/_h/v0jk_rtd6zzcy9tw3llczrf00000gn/T/ipykernel_11382/4231186573.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe_top50.loc[i , 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe_top50.loc[i, 'Price'])


,Ticker,Price,Number of Shares to Buy,One year Price Returns,One year Return Percentile,Six month Price Returns,Six month Return Percentile,Three month Price Returns,Three month Return Percentile,One month Price Returns,One month Return Percentile,HQM Score
0,VST,94.08,212,2.838433,0.998012,1.712021,0.998012,1.03857,1.0,0.428051,1.0,0.999006
1,NRG,82.3,243,1.45745,0.992048,0.740326,0.992048,0.591876,0.996024,0.167045,0.93837,0.979622
2,CEG,213.11,93,1.607484,0.994036,0.746228,0.994036,0.620097,0.998012,0.159088,0.924453,0.977634
3,HWM,82.93,241,0.844529,0.966203,0.610917,0.982107,0.314055,0.970179,0.305984,0.99006,0.977137
4,ANET,319.89,62,1.222848,0.99006,0.485511,0.970179,0.232195,0.924453,0.243789,0.984095,0.967197
5,QCOM,193.86,103,0.831287,0.962227,0.496873,0.976143,0.27573,0.95825,0.200818,0.960239,0.964215
6,GRMN,170.94,117,0.684392,0.93837,0.438988,0.93837,0.399473,0.988072,0.214753,0.974155,0.959742
7,APH,131.91,151,0.756417,0.956262,0.479281,0.968191,0.263513,0.94831,0.189665,0.952286,0.956262
8,WAB,168.32,118,0.727954,0.952286,0.455804,0.95825,0.252381,0.946322,0.166701,0.936382,0.94831
9,MU,125.29,159,0.837905,0.964215,0.593615,0.980119,0.552348,0.994036,0.11936,0.817097,0.938867


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [123]:
writer = pd.ExcelWriter('Momentum_Strategy.xlsx', engine ='xlsxwriter')
hqm_dataframe_top50.to_excel(writer, sheet_name = 'Momentum_Strategy', index= False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [124]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [125]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price',dollar_template],
    'C': ['Number of Shares to Buy',integer_template],
    'D': ['One year Price Returns',percent_template],
    'E': ['One year Return Percentile',percent_template],
    'F': ['Six month Price Returns',percent_template],
    'G': ['Six month Return Percentile',percent_template],
    'H': ['Three month Price Returns',percent_template],
    'I': ['Three month Return Percentile',percent_template],
    'J': ['One month Price Returns',percent_template],
    'K': ['One month Return Percentile',percent_template],
    'L': ['HQM Score',percent_template]
}

for cloumn in column_formats.keys():
    writer.sheets['Momentum_Strategy'].set_column(f'{cloumn}:{cloumn}', 25 , column_formats[cloumn][1])
    writer.sheets['Momentum_Strategy'].write(f'{cloumn}1', column_formats[cloumn][0], column_formats[cloumn][1])
writer.close()

## Saving Our Excel Output